# Changing object values
IfcOpenShell is not only good for reading out existing information. It also lets you manipulate information and save the changes to a file. Here is a script that overwrites all properties in a selected object with random gibberish

In [1]:
import uuid
import time
import tempfile
import ifcopenshell
m = ifcopenshell.open("data/hello_reiff_2021.ifc")
from utils.JupyterIFCRenderer import JupyterIFCRenderer
viewer = JupyterIFCRenderer(m, size=(400,300))
viewer

In [3]:
selection = viewer.getSelectedProduct()
print(selection)
if selection:
    #get the IfcProduct that is stored in the global variable 'selection'
    obj = selection
    for relDefinesByProperties in obj.IsDefinedBy:
         print("[{0}]".format(relDefinesByProperties.RelatingPropertyDefinition.Name))
         for prop in relDefinesByProperties.RelatingPropertyDefinition.HasProperties:
             print ("{:<20} :{} ".format(prop.Name,prop.NominalValue.wrappedValue))
             if type(prop.NominalValue.wrappedValue) == str:
                 prop.NominalValue.wrappedValue = "Мое новое значение"
             if type(prop.NominalValue.wrappedValue) == float:
                 prop.NominalValue.wrappedValue = 75
             if type(prop.NominalValue.wrappedValue) == bool:
                 prop.NominalValue.wrappedValue = False
# Be careful not to overwrite your original file!             
m.write(r'data/changed-properties.ifc')

None


## creating information
New information can be generated by using the `model.create...` functions that can be called for any entity instance you would like to add to your model.
Here is a simple example how to add custom properties in a custom property set to the object currently selected in the viewer

In [4]:
if selection:
    product = selection
    prop_set_guid = ifcopenshell.guid.compress(uuid.uuid1().hex)
    property_values = [
    m.createIfcPropertySingleValue("Чат", "Описание чата", m.create_entity("IfcText", "https://blenderbim.org/docs-python/ifcopenshell-python/code_examples.html"), None),
    m.createIfcPropertySingleValue("Работы оплачены", "whether or not this is a made up valeue", m.create_entity("IfcBoolean", True), None),
    ]
    property_set = m.createIfcPropertySet(prop_set_guid, product.OwnerHistory, "Мой набор свойств", None, property_values)
    rel_guid = ifcopenshell.guid.compress(uuid.uuid1().hex)
    m.createIfcRelDefinesByProperties(rel_guid, product.OwnerHistory, None, None, [product], property_set)

m.write(r'data/added_properties_added.ifc') 